<a href="https://colab.research.google.com/github/ChanceDurr/DS-Unit-2-Linear-Models/blob/master/module3-understanding-linear-regression/Unit_2_Sprint_1_Module_3_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import New York Rent Data
LOCAL = '../data/nyc/nyc-rent-2016.csv'
WEB = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/nyc/nyc-rent-2016.csv'

import pandas as pd
import numpy as np
df = pd.read_csv(WEB)
assert df.shape == (48300, 34)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import matplotlib.pyplot as plt

# Create a column for the total perks
perk_cols = ['elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
             'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
             'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
             'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
             'swimming_pool', 'new_construction', 'exclusive', 'terrace', 
             'loft', 'garden_patio', 'common_outdoor_space', 
             'wheelchair_access']

df['perk_count'] = df[perk_cols].sum(axis=1)

# Encode the interest_level column with 1, 2, and 3
int_level = {'low': 1, 'medium': 2, 'high': 3}
df['interest_level'] = df['interest_level'].replace(int_level)

In [0]:
# Change Created column to a datetime
df['created'] = pd.to_datetime(df['created'])

# Test Train Split
train = df[df['created'].dt.month < 6]
test = df[df['created'].dt.month == 6]

In [0]:
features = ['perk_count', 'interest_level', 'bedrooms',
            'bathrooms', 'latitude', 'longitude']
target = 'price'

# Create model and fit training data to it
model = LinearRegression()
model.fit(train[features], train[target])

# Get predictions from test data with model
y_pred = model.predict(test[features])

In [53]:
coef = model.coef_
intercept = model.intercept_

print(coef, intercept)

[    48.58228691   -419.2882721     478.42450629   1296.74145813
    734.68582269 -13504.69523713] -1027371.8842718429


In [0]:
# MAE
mae = mean_absolute_error(test['price'], y_pred)

In [0]:
r2 = r2_score(test['price'], y_pred)

In [0]:
mse = mean_squared_error(test['price'], y_pred)

In [0]:
smse = np.sqrt(mse)

In [66]:
print('Mean Absolute Error: ', mae)
print('R^2 Score: ', r2)
print('Mean Squared Error: ', mse)
print('Root Mean Error: ', smse)

Mean Absolute Error:  621.7039693993939
R^2 Score:  0.6130374368484142
Mean Squared Error:  765981.1725294833
Root Mean Error:  875.2035034947492


In [0]:
# https://stackoverflow.com/a/47230966
# Plotly notebook mode with google colaboratory
# You need to define this function
# And call it in each offline plotting cell

def configure_plotly_browser_state():
    import IPython
    display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [60]:
import itertools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

def viz3D(fitted_model, X, features, target='', num=100):
    """
    Visualize model predictions in 3D, for regression model fit on 2 features
    
    Parameters
    ----------
    fitted_model : scikit-learn model, already fitted
    X : pandas dataframe, which was used to fit model
    features : list of strings, column names of the 2 features
    target : string, name of target
    num : int, number of grid points for each feature
    
    References
    ----------
    https://plot.ly/python/3d-charts/
    """
    feature1, feature2 = features
    min1, max1 = X[feature1].min(), X[feature1].max()
    min2, max2 = X[feature2].min(), X[feature2].max()
    x1 = np.linspace(min1, max1, num)
    x2 = np.linspace(min2, max2, num)
    combos = list(itertools.product(x1, x2))
    Z = fitted_model.predict(combos).reshape(num, num)
    
    configure_plotly_browser_state()
    data = [go.Surface(x=x1, y=x2, z=Z)]
    layout = go.Layout(
        scene={'xaxis': {'title': feature1, 'range': [min1,max1], 'showticklabels': True}, 
               'yaxis': {'title': feature2, 'range': [min2,max2], 'showticklabels': True}, 
               'zaxis': {'title': target, 'showticklabels': True}}, 
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [61]:
# TODO
features = ['perk_count', 'interest_level']
X = df[features]

viz3D(model, X, features, target)

ValueError: ignored